<a href="https://colab.research.google.com/github/sebaaaap/iot-lab_experimentos/blob/main/analisis_topologia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Primer bloque de código




In [ ]:

# — Instala dependencias
!pip install google-cloud-storage networkx matplotlib


# — Autenticación en Colab (usa tu cuenta con permisos GCS)
from google.colab import auth
auth.authenticate_user()


# — Imports
from google.cloud import storage
import sqlite3
import json
import networkx as nx
import matplotlib.pyplot as plt


# — Parámetros de GCS y fichero local
GCS_BUCKET    = 'fit-iot-eit-udp'
GCS_BLOB_NAME = 'mg-experiments/results/owsn-20250728-022945/topologia/iotlab.db'
LOCAL_DB_FILE = 'iotlab.db'

# — Descarga de la base de datos desde GCS
client = storage.Client()
bucket = client.bucket(GCS_BUCKET)
blob   = bucket.blob(GCS_BLOB_NAME)
blob.download_to_filename(LOCAL_DB_FILE)
print(f'Descargado “{GCS_BLOB_NAME}” desde bucket “{GCS_BUCKET}” → “{LOCAL_DB_FILE}”')


# — Conexión a SQLite y lectura de la tabla
conn = sqlite3.connect(LOCAL_DB_FILE)
cur  = conn.cursor()
rows = list(cur.execute("SELECT red, fecha FROM network ORDER BY fecha"))


# — Variables de control para detectar cambios de topología
prev_edges = None
snapshot   = 0


# — Iterar sobre cada “foto” y dibujar solo si cambia la topología
for red_json, fecha in rows:
    # Parsear el JSON de aristas
    if not red_json:
        current_edges = frozenset()
    else:
        try:
            edge_list = json.loads(red_json)
            current_edges = frozenset(
                tuple(sorted((e['u'], e['v'])))
                for e in edge_list
                if 'u' in e and 'v' in e
            )
        except json.JSONDecodeError:
            continue

    # Si la topología no cambió, seguimos
    if current_edges == prev_edges:
        continue

    # Nuevo snapshot → graficar
    snapshot += 1
    G = nx.Graph()
    G.add_edges_from(current_edges)

    pos = nx.spring_layout(G, seed=42)
    plt.figure(figsize=(6, 5))
    nx.draw_networkx_nodes(G, pos, node_size=300)
    nx.draw_networkx_edges(G, pos)
    nx.draw_networkx_labels(G, pos, font_size=8)
    plt.title(f"Snapshot {snapshot} — {fecha}")
    plt.axis('off')
    plt.tight_layout()
    plt.show()

    prev_edges = current_edges


# Segundo bloque de código

In [ ]:

# Módulo 1: Análisis de Estructura de Datos


# — Importaciones básicas
import sqlite3
import json
from pprint import pprint


# — Función para inspeccionar la base de datos
def inspect_database_structure(db_path):
    """Analiza la estructura de la base de datos SQLite"""
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()

    # 1. Listar todas las tablas
    tables = cur.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
    print("Tablas encontradas:")
    pprint(tables)

    # 2. Para cada tabla, mostrar su estructura
    for table in tables:
        table_name = table[0]
        print(f"\nEstructura de la tabla '{table_name}':")

        # Obtener información de columnas
        columns = cur.execute(f"PRAGMA table_info({table_name});").fetchall()
        for col in columns:
            print(f"  - {col[1]}: {col[2]}")  # nombre y tipo de columna

        # Mostrar conteo de registros
        count = cur.execute(f"SELECT COUNT(*) FROM {table_name};").fetchone()[0]
        print(f"  Total registros: {count}")

    conn.close()


# — Función para analizar el contenido de la tabla networkx
def analyze_network_table(db_path, sample_size=3):
    """Analiza el contenido y estructura de la tabla network"""
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()

    print("\nAnálisis detallado de la tabla 'network':")

    # 1. Mostrar muestras de datos
    print(f"\nPrimeras {sample_size} filas completas:")
    for row in cur.execute(f"SELECT * FROM network LIMIT {sample_size};"):
        pprint(row)

    # 2. Analizar campo 'red' (JSON)
    print("\nAnálisis del campo JSON 'red':")
    json_samples = cur.execute(f"""
        SELECT red, fecha
        FROM network
        WHERE red IS NOT NULL AND red != ''
        LIMIT {sample_size};
    """).fetchall()

    for i, (red_json, fecha) in enumerate(json_samples, 1):
        print(f"\nMuestra JSON #{i} (Fecha: {fecha}):")
        try:
            data = json.loads(red_json)
            print("Tipo de estructura:", type(data))

            if isinstance(data, list):
                print("Número de elementos:", len(data))
                if data:
                    print("\nEjemplo de elemento:")
                    pprint(data[0])
                    print("\nCampos disponibles en los elementos:")
                    pprint(list(data[0].keys()))
        except json.JSONDecodeError:
            print("¡Error decodificando JSON!")
            print("Contenido crudo:", red_json)

    # 3. Estadísticas básicas del campo JSON
    print("\nEstadísticas del campo 'red':")
    total = cur.execute("SELECT COUNT(*) FROM network;").fetchone()[0]
    with_json = cur.execute("SELECT COUNT(*) FROM network WHERE red IS NOT NULL AND red != '';").fetchone()[0]
    print(f"- {with_json} de {total} registros tienen datos JSON ({with_json/total:.1%})")

    conn.close()

# — Función para analizar fechas y distribución temporal
def analyze_temporal_distribution(db_path):
    """Analiza la distribución temporal de los registros"""
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()

    print("\nAnálisis de distribución temporal:")

    # 1. Rango de fechas
    min_date, max_date = cur.execute("""
        SELECT MIN(fecha), MAX(fecha) FROM network;
    """).fetchone()
    print(f"\nRango temporal: De {min_date} a {max_date}")

    # 2. Frecuencia de registros (ejemplo: por hora)
    print("\nCantidad de registros por intervalo de tiempo:")
    time_stats = cur.execute("""
        SELECT strftime('%Y-%m-%d %H:00:00', fecha) as hour,
               COUNT(*)
        FROM network
        GROUP BY hour
        ORDER BY hour;
    """).fetchall()

    for hour, count in time_stats[:5]:  # Mostrar solo primeros 5
        print(f"- {hour}: {count} registros")
    if len(time_stats) > 5:
        print(f"... y {len(time_stats)-5} intervalos más")

    conn.close()

# — Función principal de análisis
def full_data_analysis(db_path):
    """Ejecuta todo el análisis de estructura de datos"""
    print("=== ANÁLISIS DE ESTRUCTURA DE DATOS ===")
    inspect_database_structure(db_path)
    analyze_network_table(db_path)
    analyze_temporal_distribution(db_path)
    print("\nAnálisis completado.")

# — Ejecutar análisis completo
full_data_analysis(LOCAL_DB_FILE)

# Tercer bloque de código


In [ ]:

# # Detección de Momentos Claves en la Topología (Versión Mejorada)
# Este notebook identifica automáticamente los momentos importantes en la evolución de la red,
# considerando la estructura específica de los datos analizados.

# — Imports
from collections import defaultdict
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import json
from datetime import datetime

# — Función mejorada para analizar cambios en la topología
def detect_key_moments_enhanced(rows):
    """
    Detecta momentos clave en la evolución de la topología de red.
    Versión mejorada para manejar casos con JSON vacío y optimizar detección.
    """
    key_moments = []
    prev_topology = None
    snapshot = 0

    for i, (red_json, fecha) in enumerate(rows):
        # Inicializar estructura para la topología actual
        current_topology = {
            'edges': set(),
            'nodes': set(),
            'parents': {},
            'timestamp': fecha
        }

        # Procesar JSON solo si no está vacío
        if red_json and red_json != '[]':
            try:
                edge_list = json.loads(red_json)

                # Procesar cada conexión
                for e in edge_list:
                    if 'u' in e and 'v' in e:
                        # Crear arista no dirigida (ordenada)
                        edge = tuple(sorted((e['u'], e['v'])))
                        current_topology['edges'].add(edge)
                        current_topology['nodes'].update([e['u'], e['v']])

                        # Detectar relaciones padre-hijo (si existe información)
                        if 'parent' in e:
                            if e['parent'] == e['u']:
                                current_topology['parents'][e['v']] = e['u']
                            else:
                                current_topology['parents'][e['u']] = e['v']
            except json.JSONDecodeError:
                print(f"Error decodificando JSON en fila {i}")
                continue

        # Si es el primer registro, registrar como inicio
        if prev_topology is None:
            snapshot += 1
            key_moments.append(create_moment_record(
                snapshot=snapshot,
                fecha=fecha,
                event_type='Inicio',
                event_detail='Primera topología registrada',
                current=current_topology,
                previous=None
            ))
            prev_topology = current_topology
            continue

        # Calcular cambios respecto a la topología anterior
        changes = calculate_topology_changes(current_topology, prev_topology)

        # Determinar si es un momento clave
        is_key_moment, event_type, event_detail = assess_key_moment(changes, prev_topology)

        # Registrar momento clave si corresponde
        if is_key_moment:
            snapshot += 1
            key_moments.append(create_moment_record(
                snapshot=snapshot,
                fecha=fecha,
                event_type=event_type,
                event_detail=event_detail,
                current=current_topology,
                previous=prev_topology
            ))

        prev_topology = current_topology

    return key_moments

def create_moment_record(snapshot, fecha, event_type, event_detail, current, previous):
    """Crea un registro estandarizado de momento clave"""
    record = {
        'snapshot': snapshot,
        'fecha': fecha,
        'tipo': event_type,
        'detalle': event_detail,
        'edges': current['edges'],
        'nodes': current['nodes'],
        'parents': current['parents']
    }

    if previous is not None:
        record.update({
            'edge_changes': current['edges'].symmetric_difference(previous['edges']),
            'node_changes': {
                'added': list(current['nodes'] - previous['nodes']),
                'removed': list(previous['nodes'] - current['nodes'])
            },
            'parent_changes': find_parent_changes(current['parents'], previous['parents'])
        })

    return record

def calculate_topology_changes(current, previous):
    """Calcula los cambios entre dos topologías consecutivas"""
    return {
        'edge_changes': current['edges'].symmetric_difference(previous['edges']),
        'node_changes': {
            'added': current['nodes'] - previous['nodes'],
            'removed': previous['nodes'] - current['nodes']
        },
        'parent_changes': find_parent_changes(current['parents'], previous['parents'])
    }

def find_parent_changes(current_parents, previous_parents):
    """Identifica cambios en las relaciones padre-hijo"""
    changes = {}
    all_nodes = set(previous_parents.keys()).union(set(current_parents.keys()))

    for node in all_nodes:
        old_parent = previous_parents.get(node)
        new_parent = current_parents.get(node)
        if old_parent != new_parent:
            changes[node] = {'old': old_parent, 'new': new_parent}

    return changes

def assess_key_moment(changes, previous_topology):
    """Evalúa si los cambios constituyen un momento clave"""
    # 1. Cambio significativo en conexiones (>30% o mínimo 2 conexiones)
    edge_change_count = len(changes['edge_changes'])
    prev_edge_count = len(previous_topology['edges'])

    if edge_change_count >= max(2, prev_edge_count * 0.3):
        return True, "Cambio mayor", f"Cambio en {edge_change_count}/{prev_edge_count} conexiones"

    # 2. Cambio en relaciones padre-hijo
    if changes['parent_changes']:
        changed_nodes = list(changes['parent_changes'].keys())
        return True, "Cambio de padres", f"Nodos con cambio de padre: {', '.join(changed_nodes)}"

    # 3. Aparición/desaparición de nodos
    node_changes = []
    if changes['node_changes']['added']:
        node_changes.append(f"Aparecieron: {', '.join(changes['node_changes']['added'])}")
    if changes['node_changes']['removed']:
        node_changes.append(f"Desaparecieron: {', '.join(changes['node_changes']['removed'])}")

    if node_changes:
        return True, "Cambio de nodos", "; ".join(node_changes)

    # 4. No es momento clave caso borde
    return False, "", ""

# — Función mejorada de visualización
def visualize_moment_enhanced(moment):
    """Visualiza un momento clave con información mejorada"""
    G = nx.Graph()
    G.add_edges_from(moment['edges'])

    # Configurar colores y estilos
    node_colors = []
    edge_colors = []
    edge_widths = []

    # Procesar nodos
    for node in G.nodes():
        if 'node_changes' in moment:
            if node in moment['node_changes']['added']:
                node_colors.append('green')  # Nodos nuevos
            elif node in moment['node_changes']['removed']:
                node_colors.append('gray')  # Nodos que desaparecerán
            elif node in moment.get('parent_changes', {}):
                node_colors.append('red')  # Nodos con cambio de padre
            elif any(node in edge for edge in moment.get('edge_changes', set())):
                node_colors.append('orange')  # Nodos con cambio de conexión
            else:
                node_colors.append('skyblue')  # Nodos sin cambios
        else:
            node_colors.append('skyblue')  # Primer momento

    # Procesar aristas
    for edge in G.edges():
        edge_tuple = tuple(sorted(edge))
        if 'edge_changes' in moment and edge_tuple in moment['edge_changes']:
            edge_colors.append('red')
            edge_widths.append(2)
        else:
            edge_colors.append('black')
            edge_widths.append(1)

    # Dibujar el grafo
    pos = nx.spring_layout(G, seed=42)  # Layout consistente
    plt.figure(figsize=(10, 8))

    nx.draw_networkx_nodes(
        G, pos,
        node_size=500,
        node_color=node_colors,
        alpha=0.9
    )

    nx.draw_networkx_edges(
        G, pos,
        edge_color=edge_colors,
        width=edge_widths,
        alpha=0.7
    )

    nx.draw_networkx_labels(
        G, pos,
        font_size=9,
        font_weight='bold'
    )

    # Configurar título informativo
    title_lines = [
        f"Momento {moment['snapshot']} — {moment['fecha']}",
        f"Tipo: {moment['tipo']}",
        f"Detalle: {moment['detalle']}",
        f"Nodos: {len(moment['nodes'])} | Aristas: {len(moment['edges'])}"
    ]

    plt.title("\n".join(title_lines), pad=20)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# — Procesar la base de datos y detectar momentos clave
key_moments = detect_key_moments_enhanced(rows)

# — Mostrar tabla resumen mejorada
if key_moments:
    df_moments = pd.DataFrame(key_moments)[['snapshot', 'fecha', 'tipo', 'detalle']]
    print("Momentos clave detectados:")

    # Formatear mejor la visualización
    pd.set_option('display.max_colwidth', 50)
    display(df_moments.style.set_properties(**{
        'text-align': 'left',
        'white-space': 'pre-wrap'
    }))

    # Visualizar el primer momento clave
    print("\nVisualización del primer momento clave:")
    visualize_moment_enhanced(key_moments[0])
else:
    print("No se detectaron momentos clave en la topología.")

# — Función para explorar momentos clave por timestamp
def explore_by_timestamp(key_moments, target_timestamp=None):
    """Permite explorar momentos clave por timestamp aproximado"""
    if not key_moments:
        print("No hay momentos clave registrados")
        return

    # Si no se especifica timestamp, mostrar lista
    if target_timestamp is None:
        print("Momentos clave disponibles:")
        for i, moment in enumerate(key_moments):
            print(f"{i+1}. {moment['fecha']} - {moment['tipo']}: {moment['detalle']}")
        return

    # Buscar el momento más cercano al timestamp especificado aqui aproxima
    try:
        target_dt = datetime.strptime(target_timestamp, '%Y-%m-%d %H:%M:%S.%f')
        closest = min(
            key_moments,
            key=lambda x: abs(datetime.strptime(x['fecha'], '%Y-%m-%d %H:%M:%S.%f') - target_dt)
        )

        print(f"\nMomento más cercano a {target_timestamp}:")
        print(f"- Snapshot: {closest['snapshot']}")
        print(f"- Fecha exacta: {closest['fecha']}")
        print(f"- Tipo: {closest['tipo']}")
        print(f"- Detalle: {closest['detalle']}")

        visualize_moment_enhanced(closest)
    except ValueError:
        print("Formato de timestamp inválido. Use: 'YYYY-MM-DD HH:MM:SS.FFFFFF'")


## Uso para este tercer bloque:


In [ ]:
explore_by_timestamp(key_moments)

In [ ]:
explore_by_timestamp(key_moments, '2025-07-28 02:36:57.816768')

# Cuarto bloque de código


In [ ]:
def analyze_moment_detail(key_moments, snapshot_num):
    """Provee análisis detallado de un momento clave específico"""
    if not 1 <= snapshot_num <= len(key_moments):
        print(f"Snapshot {snapshot_num} no existe")
        return

    moment = key_moments[snapshot_num-1]

    print(f"\n📊 Análisis detallado - Snapshot {moment['snapshot']}")
    print(f"🕒 Fecha: {moment['fecha']}")
    print(f"🏷️ Tipo: {moment['tipo']}")
    print(f"📝 Detalle: {moment['detalle']}")

    # Análisis de nodos
    nodes = moment['nodes']
    print(f"\n🔘 Nodos ({len(nodes)}): {', '.join(sorted(nodes))}")

    # Análisis de conexiones
    edges = moment['edges']
    print(f"\n🔗 Conexiones ({len(edges)}):")
    for u, v in sorted(edges):
        print(f"- {u} ↔ {v} (padre: {moment['parents'].get(v, u)})")

    # Cambios detectados
    if 'edge_changes' in moment:
        print("\n🔄 Cambios respecto al estado anterior:")
        added = [e for e in edges if e in moment['edge_changes']]
        removed = [e for e in moment['edge_changes'] if e not in edges]

        print(f"- Conexiones añadidas ({len(added)}):")
        for e in added:
            print(f"  {e[0]} ↔ {e[1]}")

        print(f"- Conexiones eliminadas ({len(removed)}):")
        for e in removed:
            print(f"  {e[0]} ↔ {e[1]}")

    # Estructura de la red
    G = nx.Graph(list(edges))
    degrees = dict(G.degree())
    max_degree = max(degrees.values()) if degrees else 0

    print(f"\n🌐 Estructura de red:")
    print(f"- Nodo(s) central(es) (grado {max_degree}): {[n for n, d in degrees.items() if d == max_degree]}")
    print(f"- Hojas (grado 1): {[n for n, d in degrees.items() if d == 1]}")
    print(f"- Aislamiento: {len([n for n, d in degrees.items() if d == 0])} nodos desconectados")

## Uso

In [ ]:
analyze_moment_detail(key_moments, 5)